<a href="https://colab.research.google.com/github/ManuSanchez02/7506R-2c2022-GRUPO09/blob/main/tp2/7506R_TP2_GRUPO09_ENTREGA_N1(nlp).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn import metrics

!pip install xgboost

np.random.seed(1)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [116]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [117]:
dataset_descrip = pd.read_csv("/content/drive/MyDrive/OrgaTP2/properati_argentina_2021_decrip.csv")

dataset_train = pd.read_csv("https://github.com/ManuSanchez02/7506R-2c2022-GRUPO09/blob/main/tp2/datasets/dataset_train_preprocesado.csv?raw=True")
dataset_test = pd.read_csv("https://github.com/ManuSanchez02/7506R-2c2022-GRUPO09/blob/main/tp2/datasets/datasetc_test_preprocesado.csv?raw=True")

In [118]:
dataset_train.drop(["Unnamed: 0"], axis = 1, inplace=True)
dataset_test.drop(["Unnamed: 0"], axis = 1, inplace=True)

In [119]:
dataset_descrip.head()

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
1,zuHOGgUE0UB71W7m/YCLoA==,¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!<br>APTO ...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
4,qg4m3QHcuo1+wMmyp7kx3Q==,Ph 2 Amb. Villa Crespo exclusivo uso profesion...


In [120]:
dataset_pln_train = dataset_train.copy()
dataset_pln_test = dataset_test.copy()

In [121]:
dataset_pln_train = dataset_pln_train.merge(dataset_descrip, left_on='id', right_on='id')
dataset_pln_test = dataset_pln_test.merge(dataset_descrip, left_on='id', right_on='id')
dataset_pln_train.head()

,id,latitud,longitud,barrio,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_description
0,/KS9ys5/j+tOi7ZtBH0beg==,-34.554271,-58.457668,Belgrano,Departamento,2,1,59,59,178000,Hermoso departamento 2 ambientes piso alto a e...
1,F5QJsmufcrReRAHbc84lxQ==,-34.613544,-58.368072,Monserrat,Departamento,3,2,71,71,129000,Corredor Responsable: Cristian Arnal Ponti - C...
2,MJP64X970Gj+Zdvn592RWA==,-34.571020,-58.433125,Palermo,Departamento,2,1,79,59,249000,Corredor Responsable: Ariel Champanier - CUCIC...
3,SEUBJTexychILiuTsn9L3Q==,-34.597384,-58.405139,Barrio Norte,Departamento,4,3,91,85,165000,SUPER OPORTUNIDAD! AL MEJOR VALOR DE M2 DEL M...
4,gQdKfHVQXAfnza6dQte4ng==,-34.591646,-58.418064,Palermo,Departamento,1,1,37,32,105000,Corredor Responsable: Hernan Perrone CUCICBA 2...


In [122]:
def normalizar(descripcion):
  descripcion = descripcion.lower()
  descripcion = descripcion.replace('á', 'a')
  descripcion = descripcion.replace('é', 'e')
  descripcion = descripcion.replace('í', 'i')
  descripcion = descripcion.replace('ó', 'o')
  descripcion = descripcion.replace('ú', 'u')
  return descripcion

In [123]:
ignorar = ["a", "ante", "cabe", "con", "de", "desde", "durante", "en", "entre", "hacia", "hasta", "mediante", "para", "por", "según", "sin", "so", "sobre", "tras", "versus", "via", "el", "la", "los", "del", "las", "los", "etc", "es", "era", "br", "y", "que", "al", "o", "un", "ba", "una", "se", "", "cocina", "ambientes", "piso", "pisos", "propiedad", "son", "muy", "esta", "dos", "cuadras"]

def collect(dataset):
  cantidades = {}
  for descripcion in dataset['property_description']:
    descripcion = normalizar(descripcion)
    palabras = re.split('[^a-z]', descripcion)
    for palabra in palabras:
      if not palabra in ignorar and len(palabra) > 2:
        cantidades[palabra] = cantidades.get(palabra, 0) + 1
  return cantidades

diccionario = collect(dataset_pln_train)

In [124]:
sorted_dic = sorted(diccionario.items(), key=lambda x: x[1], reverse=True)
print(sorted_dic[:20])

[('balcon', 65617), ('comedor', 60511), ('living', 58609), ('departamento', 55953), ('edificio', 51346), ('completo', 45404), ('dormitorio', 44593), ('expensas', 42030), ('frente', 41990), ('excelente', 40560), ('valor', 40105), ('corredor', 39518), ('inmueble', 38849), ('medidas', 38558), ('responsable', 36159), ('amplio', 34033), ('cuenta', 32367), ('placard', 32240), ('casa', 29942), ('luminoso', 29466)]


In [125]:
aspectos = [aspecto[0] for aspecto in sorted_dic[:20]]

def agregar_aspectos(dataset):
  for aspecto in aspectos:
    dataset[aspecto] = dataset["property_description"].apply(contiene_aspecto, args=(aspecto,))

def contiene_aspecto(descripcion, aspecto):
  descripcion = normalizar(descripcion)
  return int(None != re.search(aspecto, descripcion))


agregar_aspectos(dataset_pln_train)
agregar_aspectos(dataset_pln_test)

dataset_pln_train.head()

,id,latitud,longitud,barrio,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,...,valor,corredor,inmueble,medidas,responsable,amplio,cuenta,placard,casa,luminoso
0,/KS9ys5/j+tOi7ZtBH0beg==,-34.554271,-58.457668,Belgrano,Departamento,2,1,59,59,178000,...,0,0,0,0,0,1,0,0,0,0
1,F5QJsmufcrReRAHbc84lxQ==,-34.613544,-58.368072,Monserrat,Departamento,3,2,71,71,129000,...,1,1,1,0,1,1,1,1,1,0
2,MJP64X970Gj+Zdvn592RWA==,-34.571020,-58.433125,Palermo,Departamento,2,1,79,59,249000,...,1,1,0,0,1,0,1,0,1,1
3,SEUBJTexychILiuTsn9L3Q==,-34.597384,-58.405139,Barrio Norte,Departamento,4,3,91,85,165000,...,1,0,1,1,0,1,1,0,0,1
4,gQdKfHVQXAfnza6dQte4ng==,-34.591646,-58.418064,Palermo,Departamento,1,1,37,32,105000,...,1,1,1,1,1,0,0,0,1,0


## XGBoost

In [132]:
def imprimir_metricas_regresion(target_test, precios_predichos):
    # Mean Square Error
    mse = metrics.mean_squared_error(
        y_true=target_test, y_pred=precios_predichos, squared=True
    )

    print(f"El error (mse) de test es: {mse}")

    # Root Mean Square Error
    rmse = metrics.mean_squared_error(
        y_true=target_test, y_pred=precios_predichos, squared=False
    )

    print(f"El error (rmse) de test es: {rmse}")

    r2 = metrics.r2_score(target_test, precios_predichos)
    print(f"El score R2 es: {r2}")


In [128]:
columnas = ["property_price", "id", "property_description"]

def build_target_and_dataset(dataset):
  target = dataset["property_price"]
  dataset.drop(columns=columnas, inplace=True)
  dataset = pd.get_dummies(dataset, drop_first=True)
  return target, dataset

(target_train, dataset_train) = build_target_and_dataset(dataset_pln_train)
(target_test, dataset_test) = build_target_and_dataset(dataset_pln_test)

In [133]:
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators= 500, min_child_weight= 6, max_depth= 12, learning_rate= 0.07)
model = xgb.fit(dataset_train, target_train)

[21:09:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [134]:
precios_predichos_xgb = model.predict(dataset_test)
imprimir_metricas_regresion(target_test, precios_predichos_xgb)

El error (mse) de test es: 1101168426.2508464
El error (rmse) de test es: 33183.857916927715
El score R2 es: 0.8963426552761485
